In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import dash
import openpyxl
import os
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

##Iniciando o processo dados
navegador.get('https://statusinvest.com.br/acoes/eua/busca-avancada')
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[1]/div/div/div/button[2]').click()
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div[2]/a/span').click()
time.sleep(5)

#Fecha o navegador
navegador.quit()

## Renomeia o arquivo e move para pasta desejada
source = '/Users/scheiner/Downloads/statusinvest-busca-avancada.csv'
dest = '/Users/scheiner/Documents/GitHub/StatusInvest/international.csv'
os.rename(source, dest) 

print("Source path renamed to destination path successfully.")

Source path renamed to destination path successfully.


In [3]:
intl = pd.read_csv('international.csv', sep=';',  decimal=',', thousands='.', encoding="utf-8" )

intl['P/L_IDEAL'] = intl[' VPA'] / intl[' LPA']
intl['UP/DOWNSIDE'] = (intl[' VPA'] / intl['PRECO'])-1
    
value_intl = intl.loc[:, ['TICKER', 'PRECO', ' VPA' , 'DY', ' LPA', 'P/L', 'P/L_IDEAL', 'UP/DOWNSIDE', 'ROE', 
                           'P/VP', 'EV/EBIT' ,'LIQ. CORRENTE', 'MARG. LIQUIDA', 'DIV. LIQ. / PATRI.']]

value_intl.fillna(value=0, inplace=True)
value_intl.rename(columns={'LIQ. CORRENTE': "LIQ.CORR" }, inplace=True)
value_intl.rename(columns={'MARG. LIQUIDA': "MARG.LIQ" }, inplace=True)
value_intl.rename(columns={'DIV. LIQ. / PATRI.': "DIV.LIQ/PL" }, inplace=True)
    
    
value_intl = value_intl.loc[value_intl['DY'] >= 0].loc[
    value_intl['P/L'] >= 0.01].loc[value_intl['P/L'] < 10].loc[
    value_intl['ROE'] >= 6].loc[
    value_intl['EV/EBIT'] >= 0.01].loc[value_intl['EV/EBIT'] <= 10].loc[
    value_intl['LIQ.CORR'] >= 0.5]

value_intl

,TICKER,PRECO,VPA,DY,LPA,P/L,P/L_IDEAL,UP/DOWNSIDE,ROE,P/VP,EV/EBIT,LIQ.CORR,MARG.LIQ,DIV.LIQ/PL
11,AAP,56.91,45.79,5.71,6.37,8.93,7.188383,-0.195396,13.92,1.24,8.76,1.26,3.37,0.59
13,AAWW,102.48,107.04,0.00,12.43,8.25,8.611424,0.044496,11.61,0.96,8.47,0.96,7.82,0.51
23,ABG,224.14,149.14,0.00,45.49,4.93,3.278523,-0.334612,30.50,1.50,5.59,1.91,6.28,1.04
57,ACI,23.26,3.47,31.51,2.70,8.61,1.285185,-0.850817,77.76,6.70,8.99,0.78,1.86,4.09
93,ADM,77.23,46.52,2.27,7.74,9.98,6.010336,-0.397643,16.63,1.66,7.32,1.63,4.15,-0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6104,YALA,5.11,2.50,0.00,0.53,9.65,4.716981,-0.510763,21.22,2.05,4.84,8.50,28.81,-1.01
6109,YGMZ,0.45,0.44,0.00,0.08,5.57,5.500000,-0.022222,18.50,1.03,4.89,1.57,6.33,0.56
6116,YPF,12.58,25.42,0.00,3.93,3.20,6.468193,1.020668,15.46,0.49,7.30,1.15,9.72,0.57
6147,ZIM,10.83,43.66,86.33,51.64,0.21,0.845469,3.031394,118.29,0.25,0.14,1.64,45.16,-0.04


In [4]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

ticker = value_intl['TICKER']
list_intl = ticker.to_list()

indicators_lis_intl = []
for intl in list_intl:

    #Criando url
    url = f"https://statusinvest.com.br/acoes/eua/{intl}"

    
    try:
        #Abrindo o navegador
        navegador.get(url)

        time.sleep(0.25)
    
        #Coletando infos
        NAME = navegador.find_element(By.XPATH, '//*[@id="main-header"]/div[2]/div/div[1]/div[1]/h1/small').text
        MIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[2]/div/div[2]/div/span[2]').text
        MAX_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[3]/div/div[2]/div/span[2]').text
        MIN_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[2]/div/div[1]/strong').text
        MAX_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[3]/div/div[1]/strong').text
        GAIN_12M = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[5]/div/div[1]/strong').text
        GAIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div[1]/div[1]/div[5]/div/div[1]/strong').text
        ORIGEM = navegador.find_element(By.XPATH, '//*[@id="company-section"]/div[1]/div[3]/div/div[2]/div/div/div/strong/span').text
        SEGMENTO = navegador.find_element(By.XPATH, '//*[@id="company-section"]/div[1]/div[4]/div[2]/div[1]/div/div/div/a/strong').text
        
        

        #Criando dict
        dicionario = {  "intl": intl,
                        "NAME": NAME,
                        "MIN_MES": MIN_MES, 
                        "MAX_MES": MAX_MES, 
                        "MIN_52S": MIN_52S, 
                        "MAX_52S": MAX_52S, 
                        "VAL_12M": GAIN_12M,
                        "VAL_MES": GAIN_MES,
                        "ORIGEM": ORIGEM,
                        "SEGMENTO": SEGMENTO
                        
                        }

        #Criando lista de dicts
        indicators_lis_intl.append(dicionario)
    except Exception as e:
        print(e)

#Fecha o navegador
navegador.quit()

In [5]:
list_intl = pd.DataFrame.from_dict(indicators_lis_intl)

list_intl = list_intl.replace('-', '')
list_intl = list_intl.replace('-%', '')
list_intl['MIN_MES'] = (list_intl['MIN_MES']).str.lstrip('$')
list_intl['MAX_MES'] = (list_intl['MAX_MES']).str.lstrip('$')
list_intl['MIN_MES'] = (list_intl['MIN_MES']).str.strip('- ')
list_intl['MAX_MES'] = (list_intl['MAX_MES']).str.strip('- ')
list_intl['VAL_12M'] = (list_intl['VAL_12M']).str.rstrip('%')
list_intl['VAL_MES'] = (list_intl['VAL_MES']).str.rstrip('%')

list_intl.to_csv('list_intl.csv')

list_intl = pd.read_csv("list_intl.csv", sep=',', decimal=',', thousands='.', encoding="utf-8" )

list_intl.fillna(value=0, inplace=True)
list_intl = list_intl.drop(['Unnamed: 0'],axis=1)

value_intl.rename(columns={'TICKER': "TICKERS" }, inplace=True)

list_intl.rename(columns={'intl': "TICKERS" }, inplace=True)

invest_intl = pd.merge(value_intl, list_intl, on='TICKERS')

invest_intl = invest_intl[['TICKERS', 'NAME', 'PRECO', 'MIN_MES', 'MAX_MES', 'VAL_MES', 'MIN_52S', 'MAX_52S',  'VAL_12M', 
                       ' VPA', 'DY', ' LPA', 'P/L' ,'P/L_IDEAL', 'UP/DOWNSIDE', 'ROE', 'P/VP', 'EV/EBIT', 
                       'LIQ.CORR', 'MARG.LIQ', 'DIV.LIQ/PL', 'ORIGEM', 'SEGMENTO']]

invest_intl = invest_intl.loc[invest_intl['DY'] >= 0].loc[
    invest_intl['P/L'] >= 0.01].loc[invest_intl['P/L'] < 10].loc[
    invest_intl['ROE'] >= 6].loc[
    invest_intl['EV/EBIT'] >= 0.01].loc[invest_intl['EV/EBIT'] <= 10].loc[
    invest_intl['LIQ.CORR'] >= 0.5].loc[
    invest_intl['UP/DOWNSIDE'] >= 0.1]

#invest_intl[(invest_intl['TICKERS'] == 'MSFT')]
#invest_intl.loc[invest_intl['SEGMENTO'] == 'Technology'].loc[invest_intl['ORIGEM'] == 'NASDAQ']
# 
invest_intl


,TICKERS,NAME,PRECO,MIN_MES,MAX_MES,VAL_MES,MIN_52S,MAX_52S,VAL_12M,VPA,...,P/L_IDEAL,UP/DOWNSIDE,ROE,P/VP,EV/EBIT,LIQ.CORR,MARG.LIQ,DIV.LIQ/PL,ORIGEM,SEGMENTO
15,AMPY,AMPLIFY ENERGY CORP.,7.14,6.60,7.50,25.93,5.67,10.21,25.93,9.17,...,0.814387,0.284314,122.81,0.78,1.95,0.87,116.64,0.33,NYSE,Energy
40,BEST,BEST INC.,2.40,2.37,2.90,245.27,0.50,3.01,245.27,10.16,...,1.946360,3.233333,51.40,0.24,2.06,0.90,44.90,-1.61,NYSE,Transportation
42,BGFV,BIG 5 SPORTING GOODS CORPORATION,6.96,6.86,7.65,-36.32,6.86,13.56,-36.32,12.11,...,6.147208,0.739943,16.29,0.57,2.19,1.61,4.30,-0.11,NASDAQ,Consumer Services
44,BH,BIGLARI HOLDINGS INC.,165.11,165.11,186.76,38.57,115.60,212.62,38.57,198.63,...,5.660587,0.203016,17.67,0.83,3.39,1.26,29.10,-0.02,NYSE,Consumer Services
48,BRID,BRIDGFORD FOODS CORPORATION,10.91,10.61,11.67,-23.71,10.61,14.92,-23.71,14.04,...,2.830645,0.286893,35.35,0.78,1.39,4.18,17.39,-0.09,NASDAQ,Consumer Non-Durables
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,X,UNITED STATES STEEL CORPORATION,32.02,30.22,32.02,71.78,18.12,32.15,71.78,48.10,...,8.003328,0.502186,12.49,0.67,5.39,1.92,7.05,0.11,NYSE,Basic Industries
403,XCUR,EXICURE INC.,0.74,0.74,0.96,-47.52,0.71,1.86,-47.52,1.12,...,3.200000,0.513514,31.58,0.66,1.24,2.56,12.88,-0.24,NASDAQ,Health Care
405,XYF,X FINANCIAL,4.18,4.06,4.52,79.78,1.62,5.00,79.78,13.52,...,5.656904,2.234450,17.71,0.31,0.21,1.89,20.98,-0.25,NYSE,0
408,YPF,YPF SOCIEDAD ANONIMA,12.58,11.99,14.20,102.58,6.21,15.67,102.58,25.42,...,6.468193,1.020668,15.46,0.49,7.30,1.15,9.72,0.57,NYSE,Energy
